In [23]:
import MySQLdb
import pandas as pd
import random
import time

from datetime import datetime,timedelta

In [24]:
#coneccion al db
db = MySQLdb.connect(host="172.18.55.6",    # tu host, usualmente localhost
                     user="comandato",         # tu usuario
                     passwd="comandato123",  # tu password
                     db="sistemecuador_atm")        # el nombre de la base de datos

In [25]:
cur = db.cursor()

In [26]:
Anio='2018'
Mes='09'
Dia_act='1'

In [27]:
#nombre de los archivos Asignacion normal
nom_actual='ESTRUCTURA DE GESTION Y MASIVOS.xlsx'
nombre_hoja='Hoja1'
cartera='CORRIENTE'

In [28]:
#leer archivo excel asignacion actual y anterior
base=pd.read_excel(open(nom_actual,'rb'), sheet_name=nombre_hoja,dtype={"Numero de Idetificacion del Cliente": str,"Numero de Telefono en el Cual se Realizo la Gestión.":str})

In [29]:
base.columns = base.columns.str.replace(' ','_')

In [30]:
base = base.rename(columns={'Numero_de_Idetificacion_del_Cliente': 'CED','DD/MM/AAAA/_hh:mm:ss':'fech_gets','Accion_Realizada_en_la_Gestión:\n*_Hacer_llamada.':'Accion','Respuesta_Obtenida_en_la_Gestión':'RESPUESTA','Comentarios_de_la_Gestión_Realizada_':'COMENTARIO','Numero_de_Telefono_en_el_Cual_se_Realizo_la_Gestión.':'TELEFONO','Fecha_en_la_Cual_se_Gestionará_el_cliente_DD/MM/AAAA/_hh:mm:ss':'fech_prox','Tiempo_en_que_tomo_en_realizar_la_Gestión':'TIEMPO',})

In [31]:
base_ivr=base[(base['Accion']=='ENVIAR IVR') | (base['Accion']=='ENVIAR SMS')]

In [32]:
base_ivr= base_ivr.fillna('')

In [ ]:
contador_ya_gestion=0
contador_subidos=0
contador=0
for row in base_ivr.itertuples():
    telefono=str(row.TELEFONO)
    cedula=str(row.CED)
    fech_gest=str(row.fech_gets)[:19]
    repuesta=str(row.RESPUESTA)
    ob_fecha=datetime.strptime(fech_gest,'%Y-%m-%d %H:%M:%S')
    fech_prox=str(ob_fecha+ timedelta(days=3))
    comentario=repuesta+' '+telefono
    usuario=str('SISTEMC'+str(random.randrange(1,28,1)).zfill(2))
    tiempo=str(row.TIEMPO)
    ACION=str(row.Accion)
    ini_querry="""INSERT INTO `sistemecuador_atm`.`gestion_ivr` (`TIPO_GESTION`, 
                                `NRO_IDENTIFICACION_CLIENTE`, `CASA_COBRANZA`,
                                `COD_AGENTE`, `USUARIO_SAC`, `FECHA_GESTION`, 
                                `ACCION_REALIZADA`, `RESPUESTA_OBTENIDA`, 
                                `CONTACTO_GESTIONO`,`COMENTARIOS_GESTIONO`,
                                `NUMERO_GESTION`, `FECHA_PROXIMA_GESTION`, 
                                 `TIEMPO_GESTION`, `CANAL_GESTION`) 
                        VALUES ('1', '{0}', 'SISTEMA DE COBRO DEL ECUADOR',
                                'D05554', 'SISTEMSUPER', '{1}', 
                                '{7}','{2}', 'TITULAR',
                                '{3}','{4}',
                                '{5}','{6}', 'GESTION');""".format(cedula,fech_gest,repuesta,comentario,telefono,fech_prox,tiempo,ACION)
    try:
        insert=cur.execute(ini_querry)
        db.commit()
        contador_subidos=contador_subidos+1
    except:
        db.rollback()
        print('falla')
    contador=contador+1
    if (contador%10000)==0:
        print(contador)
        db = MySQLdb.connect(host="172.18.55.6",    # tu host, usualmente localhost
                     user="comandato",         # tu usuario
                     passwd="comandato123",  # tu password
                     db="sistemecuador_atm")        # el nombre de la base de datos
        cur = db.cursor()
print("CANTIDAD DE CLIENTES QUE YA FUERON GESTIONADOS : "+str(contador_ya_gestion))
print("CANTIDAD DE CLIENTES QUE FUERON SUBIDOS : "+str(contador_subidos))

10000
20000
30000


In [22]:
consulta_contarivr="""
    SELECT max(idGESTION)
    FROM
        sistemecuador_atm.gestion_ivr;
"""
contador=pd.read_sql(consulta_contarivr, con=db)
contador

,max(idGESTION)
0,10330275


In [10]:
consulta_max="""
    SELECT NRO_IDENTIFICACION_CLIENTE,FECHA_GESTION
    FROM
        sistemecuador_atm.gestion_ivr
    WHERE
        idGESTION='10330274';
"""
cedmax=pd.read_sql(consulta_max, con=db)
cedmax

,NRO_IDENTIFICACION_CLIENTE,FECHA_GESTION
0,0992440414001,2019-03-08 12:59:59
